# odoo中的安全
- odoo中采用两种数据驱动的机制来管理和限制对数据的访问，两种安全机制机制都是与权限组相关联，而用户可以归属于任一权限组，从而将安全机制应用到用户。

## res.groups类:定义用户组
- name：作为用户可读的组标识（指明角色或者该权限组的作用）
- category_id：用户组所属的模块名，用于将组与Odoo App(一组相关的业务模型)关联起来，并将它们转换为用户表单中的独家选择。
    - <field name="category_id" ref="base.module_category_hidden"/>
- implied_ids：基于哪个用户组，类似伪继承
    - <field name="category_id" ref="base.module_category_hidden"/>
    - 这个ID是通过ir.module.category来定义的
            <record model="ir.module.category" id="module_category_hidden">
            <field name="name">Technical</field>
            <field name="sequence">60</field>
            <field name="visible" eval="0" />
        </record>
- users：预设用户属于这个用户组
    - <field name="users" eval="[Command.link(ref('base.user_root')), Command.link(ref('base.user_admin'))]"/>
- comments:备注
- 示例：
        <record model="res.groups" id="group_system">
            <field name="name">Settings</field>
            <field name="category_id" ref="base.module_category_hidden"/>
            <field name="implied_ids" eval="[Command.link(ref('group_erp_manager'))]"/>
            <field name="users" eval="[Command.link(ref('base.user_root')), Command.link(ref('base.user_admin'))]"/>
        </record>

## 模型访问权限：ir.model.access类
- id:可省略
- name：ir.model.access的名字
- model_id：被控制访问的模型，model_+model_name
- group_id：被授予访问权限的权限组，可省略（表示所有用户），如果填了，则是在res.groups中定义的用户组
- perm_create:
- perm_read:
- perm_write:
- perm_unlink:
- 示例：
        id,name,model_id/id,group_id/id,perm_read,perm_write,perm_create,perm_unlink
        access_test_model,access_test_model,model_test_model,base.group_user,1,0,0,0

## record访问规则：ir.rule
- ir.rule记录的规则是为了允许操作而必须满足的条件。它按照访问权限逐个记录评估。
- name：rule的描述
- model_id:规则应用的模型
- groups：授予(或不授予)访问权限的res.groups。可以指定多个组。如果没有指定组，则该规则是全局规则，其处理方式与“组”规则不同(见下文)。
    - <field name="groups" eval="[Command.link(ref('base.group_portal')), Command.link(ref('base.group_public'))]"/>
- global：
    - 基于groups进行计算的，可以方便地访问规则的全局状态(或非全局状态)
    - <field eval="False" name="global"/>
    - 关于global使用的一些说明：
        - 1.全局规则相交，如果应用两个全局规则，则必须满足两个规则才能授予访问，这意味着添加全局规则总是进一步限制访问。
        - 2.如果应用了两个组规则，那么可以满足其中任何一个规则来授予访问权限。这意味着添加组规则可以扩展访问权限，但不能超出全局规则定义的范围。
        - 3.全局规则集和组规则集相交，这意味着添加到给定全局规则集的第一个组规则将限制访问。
- domain_force:
    - 如果域与记录匹配，则规则允许所选操作，否则禁止。
    - <field name="domain_force">[('id', 'child_of', user.commercial_partner_id.id)]</field>
- perm_create:
- perm_read:
- perm_write:
- perm_unlink:
- 以上四个增删查改与ir.model.access中的完全不一样，它们指定了规则应用的操作。如果没有选择某个操作，则不会对该规则进行检查，就像该规则不存在一样。
- 示例：
            <record id="ir_rule_crm_reveal_rule_salesman" model="ir.rule">
                <field name="name">CRM Reveal Rules: Personal / Global Rules</field>
                <field name="model_id" ref="model_crm_reveal_rule"/>
                <field name="domain_force">['|', ('user_id', '=', user.id), ('user_id', '=', False)]</field>
                <field name="groups" eval="[(4, ref('sales_team.group_sale_salesman'))]"/>
            </record>
            
## 字段访问权限
- 可以在字段定义时加上一个groups参数

## 权限的概述：
一：Odoo中的权限设置主要有以下5种
    1）菜单、报表的访问权限

    Odoo可以设置菜单项、某些报表的可见性，通过在xml文件中，对应的menu标签、report标签的groups属性来定义。

    2）模型的访问权限设置

    Odoo可以设置模型对象的增删读改权限，这个在下面会详解。

    3）记录的访问权限设置

    模型的权限是针对改模型的所有记录的，而记录的访问权限则更加细致，针对每一条记录进行过滤筛选。在下文中详解。

    4）字段的访问权限设置

    Odoo可以设置具体字段的访问权限，包括可见性、写、改权限等。

    有两种方式：一种是在模型定义的代码中，通过某字段的read=[群组]、write=[群组]属性，指定字段对于哪些群组可读写。

                      另一种是在模型的视图xml文件中，对字段通过 groups属性指定访问权限。

    5）工作流操作权限

    工作流中的活动可以设置操作权限，规定只有哪些群组的用户才可以操作。

    1～4，都是通过群组来设置的，把用户添加至group，然后在代码中通过group以群组为单元进行权限设置。也可以在Odoo的开发者模式下，点击“群组”菜单，在可视化界面中进行设置：